**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 9.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Na6f48e99e98a4e39b50836a22a872571 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [3]:
#RDFLib
print("Parte de RDFLib:")
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s, "es subclase de Person.")
  for s2, p2, o2 in g.triples((None, RDFS.subClassOf, s)):
    print(s2, "es subclase de", s, "que a su vez es subclase de Person.")


#SPARQL
print("Parte de SPARQL:")
from rdflib.plugins.sparql import prepareQuery
  #PREFIX ex: <https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4>
query = prepareQuery('''
  SELECT DISTINCT ?x
  WHERE{
	  ?x (rdfs:subClassOf/rdfs:subClassOf*) ns:Person		      # normalmente sería "?x rdfs:subClassOf ex:Establishment", pero como queremos navegar no sólo por la primera línea de subclases, 
  }							                                            # sino que por toda la jerarquía, se le añade un "/subClassOf*"
  ''',
  initNs = {"rdfs": RDFS, "ns": ns}
)
for i in g.query(query):
  print(i)

Parte de RDFLib:
http://somewhere#Researcher es subclase de Person.
http://somewhere#PhDstudent es subclase de http://somewhere#Researcher que a su vez es subclase de Person.
Parte de SPARQL:
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [5]:
#RDFLib
print("Parte de RDFLib:")
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s, "es del tipo Person.")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s2, p2, o2 in g.triples((None, RDF.type, s)):
    print(s2, "es subclase de", s, "que a su vez es del tipo Person.")


#SPARQL
print("Parte de SPARQL:")
from rdflib.plugins.sparql import prepareQuery
    #PREFIX ex: <https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4>
query = prepareQuery('''
  SELECT DISTINCT ?x
  WHERE{
    {?x rdf:type ns:Person} UNION
	  {
    ?p (rdfs:subClassOf/rdfs:subClassOf*) ns:Person .
    ?x rdf:type ?p
    } 
  }
  ''',
  initNs = {"rdf": RDF, "rdfs": RDFS, "ns": ns}
)
for j in g.query(query):
  print(i)

Parte de RDFLib:
http://somewhere#SaraJones es del tipo Person.
http://somewhere#JohnSmith es del tipo Person.
http://somewhere#JaneSmith es subclase de http://somewhere#Researcher que a su vez es del tipo Person.
Parte de SPARQL:
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [6]:
#RDFLib
print("Parte de RDFLib:")
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s2, p2, o2 in g.triples((s, None, None)):
    print(s2, p2, o2)
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s2, p2, o2 in g.triples((None, RDF.type, s)):
    for s3, p3, o3 in g.triples((s2, None, None)):
      print(s3, p3, o3)


#SPARQL
print("Parte de SPARQL:")
from rdflib.plugins.sparql import prepareQuery
    #PREFIX ex: <https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4>
query = prepareQuery('''
  SELECT DISTINCT ?s ?p ?o
  WHERE{
  	{
    ?s rdf:type ?s2 .
    ?s2 (rdfs:subClassOf/rdfs:subClassOf*) ns:Person .
    ?s ?p ?o
    } UNION {
    ?s rdf:type ns:Person .
    ?s ?p ?o
    }
  }
  ''',
  initNs = {"rdf": RDF, "rdfs": RDFS, "ns": ns}
)
for k in g.query(query):
  print(i)

Parte de RDFLib:
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://so